# Search suggest autocomplete
From [Wikipedia](https://en.wikipedia.org/wiki/Search_suggest_drop-down_list):
> A search suggest drop-down list is a query feature used in computing to show the searcher shortcuts, while the query is typed into a text box. Before the query is complete, a drop-down list with the suggested completions appears to provide options to select. The suggested queries then enable the searcher to complete the required search quickly. As a form of autocompletion, the suggestion list is distinct from search history in that it attempts to be predictive even when the user is searching for the first time. Data may come from popular searches, sponsors, geographic location or other sources.

The feature exists in virtually all search interfaces of products we use every day: Google, Slack,  YouTube, etc. Here an example of "[Google Suggest](https://support.google.com/websearch/answer/106230)" in action:

![](https://drive.google.com/uc?export=view&id=1mrwSPEiputqBtwDxJIhPj4-LfbYoxOgA)

In this notebook we build a working prototype for such a feature.

## Requirements
**Input**: Partial query entered by the user, e.g. "machine lea" or "machien learning". Note that the input may have misspelled or partial words. 

**Output**: A ranked list of suggested searches, e.g. ```["machine learning interview questions", "machine learning algorithms", "machine learning engineer", ...]```. If there are a large number of suggestions generated, you may return the top 10 most relevant ones.

# Data
1. For the first part of this work, we'll use the [MSMARCO Full Web Documents](https://microsoft.github.io/msmarco/) dataset downloadable at [link](https://msmarco.blob.core.windows.net/msmarcoranking/fulldocs.tsv.gz) -- a dataset of web documents indexed by Bing. Note that this dataset does not include any user queries.
2. For the second part, we'll use the MSMARCO Queries dataset downloadable at [link](https://msmarco.blob.core.windows.net/msmarcoranking/queries.tar.gz). 




## Part 1: Search suggestions using documents
For the first user on the system, we'll face a cold-start problem where we don't have access to any past queries. Build a search suggest prototype that leverages the MSMARCO Full Web Documents dataset -- and is able to generate relevant search suggestions without using any user queries.



### **Proposed Solution:**

## **For Demo, please visit [here](http://cde01575bbdf.ngrok.io/)**

I believe there are multiple ways to approach this problem. Our Initial Approach is described below:


*   **Big Picture**: Generate Artificial queries using document passages, build meaningful graph using the generated queries, and to suggest autocomplete, traverse the query graph and identify relevant queries. 
*   **Key Advantages**: 
    * Sub 100ms latency 
    * Highly Scalable when integrated with graph databases
    * Continous learning is comparitively easier than when using DL models
    * Since the queries are generated from passages, we have a direct mapping between an autocompleted query and a document. This can be used effectively for search & relevance. 
    * inherently takes care of multi-langauge.
    * Graph structure & properties makes the approach highly flexible and customizable
    * can easily integrate with the next iteration when we have user queries.
    * can be complemented with retrievers and generators to improve search & relevance.
    * Most of the computation happens offline.
    * Direct Approach to Auto-correct
*   **Key Disadvantages**
    * lacks semantic matching during retrieval 
    * No Obvious way to rank suggested autocomplete queries unless until we have a complementing data like user feedback or user-queries.
      * Complementing with OOTB retrievers would fail due to un-clear syntatic representation weightage and not-enough information on the query
        * A similar experiment was done and found to highlight this problem during this part of solution
    



#### Step 1: Generate Artifical Queries

*  We will be using [docTTTTTquery model](https://github.com/castorini/docTTTTTquery), trained on MS MARCO Passage Ranking Dataset.
*  Before we proceed with the actual Generation, we split the articles into passages. I have not included the script since we would be directly following and using the script provided by [Colbert](https://github.com/stanford-futuredata/ColBERT/blob/master/utility/preprocess/docs2passages.py), this intern is the same implementation adopted from DPR. 

**Note: This is a very compute-intensive job! due to compute and time constrain, we generate queries only for 40225 passages in MS-marco corpus. This took aproximately 13 hours!**



In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = T5Tokenizer.from_pretrained('castorini/doc2query-t5-large-msmarco')
model = T5ForConditionalGeneration.from_pretrained('castorini/doc2query-t5-large-msmarco')
model = model.to(device)

def get_qg(doc_text):

    input_ids = tokenizer.encode(doc_text, truncation=True, max_length=512, padding="longest", return_tensors='pt').to(device)
    outputs = model.generate(
        input_ids=input_ids,
        max_length=64,
        do_sample=True,
        top_k=10,
        top_p=0,
        temperature=0.9,
        num_return_sequences=50)

    return list(set([tokenizer.decode(outputs[i], skip_special_tokens=True) for i in range(50)]))

assert len(get_qg("Hello world is a rudementary check for every programming language!"))!=0

In [ ]:
import csv
import sys
from tqdm import tqdm

csv.field_size_limit(sys.maxsize)

# Iterative through our passages and generate qeuries.
# Store queries in a txt file for further steps
with open("../data/fulldocs.tsv", "rt") as f_reader:
    tsv_reader = csv.reader(f_reader, delimiter="\t")
    with open('../data/qg.txt', 'w') as f:
        for row in tqdm(tsv_reader):
            qg_list = get_qg(row[-1])
            
            for question in qg_list:
                f.write(question)
                f.write('\n')

#### STEP 2: Query Graph Building



*   The graph connection is as follows:
  * Example sentence: ***How do I file taxes*** 
  * **How** -> **How do** -> **How do I** -> **How do I file** -> **How do I file taxes**

*  **Note**: A better graph would be to either represent them as individual tokens or even characters but the traversal, auto-correction, semantic and syntactic correction checks becomes exponentially harder. Therefore, for this prototype, I have went with a reasonable approach which sacrifies space complexity.

In [ ]:
import pandas as pd
import Levenshtein as ln
import networkx as nx
from ast import literal_eval

from tqdm import tqdm
import pickle

In [ ]:
# pre-compute question & ids map

ids_question_map = {}
question_ids_map = {}
nline = 0

with open("../data/qg.txt", "r") as f:
    with tqdm(total=1563052) as progress_bar:
        while line := f.readline():
            ids_question_map[nline] = line.strip('\n').lower()
            question_ids_map[line.strip('\n').lower()] = nline
            nline += 1
            progress_bar.update(1)

pickle.dump(ids_question_map, open("../index/ids_question_map.pkl", "wb"))
pickle.dump(question_ids_map, open("../index/question_ids_map.pkl", "wb"))

In [ ]:
ids_question_map = pickle.load(open("../index/ids_question_map.pkl", "rb"))
question_ids_map = pickle.load(open("../index/question_ids_map.pkl", "rb"))

G = nx.DiGraph()

track_leafs = {}

with open("../data/qg.txt", "r") as f:
    nlines = 0 
    with tqdm(total=1563052) as progress_bar:
        while line := f.readline():
            if line!='':
                question = line.strip("\n").lower()
                tokenized = question.split()

                if not tokenized:
                    continue

                prev_node_entry = tokenized[0]
                for i in range(2, len(tokenized)+1):

                    # Create Nodes and add property Starts, if it's the start of
                    # query
                    if len(prev_node_entry.split()) == 1:
                        if prev_node_entry not in G:
                            G.add_node(prev_node_entry, starts=True)
                        else:
                            G.nodes[prev_node_entry]['starts'] = True
                    else:
                        if prev_node_entry not in G:
                            G.add_node(prev_node_entry, starts=False)
                        else:
                            G.nodes[prev_node_entry]['starts'] = False

                    if len(" ".join(tokenized[:i]).split()) == 1:
                        if " ".join(tokenized[:i]) not in G:
                            G.add_node(" ".join(tokenized[:i]), starts=True)
                        else:
                            G.nodes[" ".join(tokenized[:i])]['starts'] = True
                    else:
                        if " ".join(tokenized[:i]) not in G:
                            G.add_node(" ".join(tokenized[:i]), starts=False)
                        else:
                            G.nodes[" ".join(tokenized[:i])]['starts'] = False

                    # pre-computing leaf nodes, so that we don't have to 
                    # find descendants during inference
                    track_leafs[prev_node_entry] = track_leafs.get(prev_node_entry, []) + question_ids_map[question]
                    
                    # Add connection between sentence[:i] -> sentence[:i+1]
                    G.add_edge(prev_node_entry, " ".join(tokenized[:i]))

                    prev_node_entry =  " ".join(tokenized[:i])

                track_leafs[question] = track_leafs.get(question, []) + question_ids_map[question]
                    
            progress_bar.update(1)

# keep_track of root_nodes. Alternate, much preferred approach is to 
# have a single root node in the graph which connects to the below nodes.
roots_list = [i for i,j in G.nodes(data="starts", default=1) if j==True]

nx.write_gpickle(G, "../index/query_graph.gpickle")
pickle.dump(track_leafs, open("../index/track_leafs.pkl", "wb"))

#### Step-3: Inference Pipeline; Traverse Graph for Query Auto-complete



In [ ]:
import timeit

# Sample Query 
query = "hwat ia the bst way to file".lower()


tokenized_query = query.split()
len_tokenized_query = len(tokenized_query)
node_list = []

def recurssive_search(comparison_term, G, idx=1, root=False, node_list=[], forked=False):
    new_term = []
    for node in (roots_list if root else G[" ".join(comparison_term.split()[:-1])]):
        
        # don't auto-correct on current-word - like google
        if ((ln.distance(comparison_term, node) <= 2) if idx!=len_tokenized_query-1 else tokenized_query[-1] in node):
            if idx!=len_tokenized_query-1:
                node_list.extend(recurssive_search(node + " " + tokenized_query[idx+1], G, idx+1, root=False, node_list=node_list, forked=True))
            else:
                if node in G and node in track_leafs:
                    new_term.append(node)

              
    if forked:
        if idx==len_tokenized_query-1:
            return [ids_question_map[child] for term in new_term for child in track_leafs[term]]
        return ""
    else:
        if len(tokenized_query)==1:
            if idx==len_tokenized_query-1:
                return [ids_question_map[child] for term in new_term for child in track_leafs[term]]
        return node_list

start_time = timeit.default_timer()
suggested_queries = recurssive_search(tokenized_query[0], G, idx=0, root=True)

# aprox. 0.07ms even for root query search like 'what' 
print("Total time taken: ", timeit.default_timer()-start_time)

#### Step-4: Semantic Ranking/retrieval *(Failed Experiment)*



*   Like discussed before, the obvious way to rank the suggested queries would be to sort by similarity between user inference search query & generated search queries 
*   We make use of Google universal sentence encoder model for encoding the representation and Faiss indexing store for comparison.
*   **Note**: Faiss indexing is again an offline process which doesn't affect our inference time.



In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import tensorflow_text

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')

tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])


embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

def get_embedding(text):
    if isinstance(text, str):
        return embed([text]).numpy()
    return embed(text).numpy()

assert get_embedding("Hello World").shape == (1, 512)

In [ ]:
import faiss
import numpy as np
from tqdm import tqdm
import pickle

D = 512

# A basic indexing scheme
index = faiss.IndexFlatL2(D)

ids_question_map = {}
nline = 0

with open("../data/qg.txt", "r") as f:
    with tqdm(total=1563052) as progress_bar:
        while line := f.readline():
            index.add(get_embedding(line.strip('\n')))
            ids_question_map[nline] = line.strip('\n')
            nline += 1
            progress_bar.update(1)

faiss.write_index(index, "../index/index.bin")
pickle.dump(ids_question_map, open("../index/ids_question_map.pkl", "wb"))

In [ ]:
# Example Inference Search 

topk = 4
query = "how do you find the"

dists, ids = index.search(x=get_embedding(query), k=1000)
accepted_queries = [(ids_question_map[q], d) for q, d in zip(ids[0], dists[0]) if ids_question_map[q] in qg_list]
sorted_accepted_queries = sorted(accepted_queries, key=lambda kv:kv[1], reverse=False)


##### The problem with this approach is that, 
*  the correlation between similarity score of encoded representation and the characteristics of the text like semantic and syntatic property is not clearly defined.
  * For example, two sentence can have high similarity because of either same intent, same context or similar grammatical structre.
*  Sudden shift in semantic and grammatical properties might off-put users during our suggestions.
  * example: query -> Machine learning is- #autocomplete#
    * suggested query -> is Artificial Intelligence usually interchangeably used with Machine learning
    * Observation: There is nothing obviously wrong with the above suggested query. It is relevant to the original query, in-regards to context. but from UI/UX point-of-view, the grammaticial change in structure might spoil the user experience and the retrieved suggestion may or may-not be relevant unless we get more input from the user.
    
**Note**: The obvious alternative to this ranking would be to incorpurate document trend similar to queries trend which we will do in the next iterative Approach

## Part 2: Search suggestions using documents and queries 
In this part we'll augment the system developed in Part 1 to use the user queries from MSMARCO Queries dataset. Build a search suggest prototype that leverages both queries and documents from our dataset -- and is able to improve on the search suggestions generated by the system developed in Part 1.


### Proposed Solution

We have to **establish couple of assumption** before we proceed on-


*   The MSMARCO Queries are considered queries from a tenant instance which contains MSMARCO documents. User-level segregation is not done.
  * If we suggest an autocomplete query to a user that is present in our MSMARCO Queries set (ie. the query was already asked in the tenant instance), we aren't suggesting redundant information to the same user but this is highly likely a **different user in the tenant space who has the same question**.
*  The **objective reasoning for relevance** would depend on the below factors:
  * The **lay syntactic and semantic similarity** between the partial user query and the suggested queries.
    * for example. For the query '**what is the best way to connect**', The two suggested responses might be '**what is the best way to connect to an Office VPN**' and other '**for VPN connection in the office, what is the best way**'. We would prefer the first one over the other due it's syntactic similarity to the query.
  * Among the suggested queries which includes both artifical queries and tenant queries. **How similar is each of these to tenant's queries trend**? 
    * ie. If the tenant space is mostly on Machine learning, when user types '**transformers**', we have to assume they are talking about awesome '**huggingface transformers**' library (or) **the architecture**, and not the '**transformers movie**'. (Assuming we have both of these topics discussed in our tenants documents) 
* Our Ranking approach needs to be flexible and extensible to include more metrics in the future like geogrophical metadata, timing window meta-data, user-role metadata, click-stream etc.
  * Once we have more of these information, the obvious step would be to try Learning-to-rank approach.


### Our Approach

*  **Big Picture:**
    *   Like we mentioned in our initial Approach, it is very feasible and reasonable to **extend the queries graph to new user queries**.
      * Eventhough, we might be tempted to, there is **no objective reason to prefer user queries more than artifically generated queries**. It should be noted that user queries would be more in-formal and diverse than our generated queries, which is inherently being prefered by algorithm rather than an explicit bias. 
    *  We use the same approach for building queries graph but this time, instead of starting from scratch, we use the existing artificially generated queries graph.
    * Like mentioned earlier, **the user queries describe a trend and topics of interest which we would like to bias our final ranking with**. To extract the trend, **we encode our user search queries, index them with FAISS and run KNN to get centroid representation, which basically is topic representation**.     
    * To get a relevance score, we find **cosine similarity between topic representations and suggested queries**. The suggested queries with higher scores are the ones which are more relevant to tenant's trend and in-turn more relevant to user (Assumption).
    * If we recall our factors for relevance, we are actually missing one factor, **suggested queries relevance to the users partial input in the search**. To handle this, instead of finding cosine similarity between topic representations and suggested queries, we find similarity between **topic representation augmented with partial input query representation** and **suggested queries**.  
      * Instead of having another direct comparison between partial input query representation and suggested queries, we augment them since **our objective is to strengthen the topic representation**.
    * One key factor to note is, we are not considering syntactic similarity between query and the suggested autocomplete. This is not implicitly taken care by cosine similarity comparison of encoded representation.
      * Therefore to add this, we bias the cosine score with Levenshtein distance, which strongly considers syntactic similarity. 
    * Like usual, we sort them based on their scores and show top N.

### Couple of things to note:
* chosing K for kNN is usually important but in-this case, we can make certain assumptions to make it non-consequential. Particularly since our dataset is massive, we choose a large enough k, such that we can atleast guarantee topic diffusion and minimize the chance for noisy cluster. 
* Again, since we are on the clock, **we encode only first 50,000 user queries** and use it for finding trend.

#### STEP 1: Encode the queries and get topic representation (Offline process)

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import tensorflow_text

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')

tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])


embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

def get_embedding(text):
    if isinstance(text, str):
        return embed([text]).numpy()
    return embed(text).numpy()

assert get_embedding("Hello World").shape == (1, 512)

In [ ]:
limit = 50000

question_ids_map = pickle.load(open("../index/question_ids_map.pkl", "rb"))
with open("../data/queries.train.tsv", "r") as f:
    tsv_reader = csv.reader(f, delimiter="\t")
    nlines = 0 
    for row in tqdm(tsv_reader, total=2371783):
        nlines += 1
        line = row[-1].lower()
        if line!='':
            X = np.vstack((X, get_embedding(line)))
        if not limit:
            break
        else:
            limit-=1

X = X[1:, :]

np.save(open("../index/v2/user_queries.npy", "wb"), X)
#X = np.load(open("../index/v2/user_queries.npy", "rb"))

In [ ]:
import faiss
import numpy as np
import pickle
import csv
from tqdm import tqdm

D = 512
K = 10
n_samples = 50000
kmeans = faiss.Kmeans(d=D, k=round(16*(n_samples**(1/2))), niter=20, verbose=True, gpu=True)

X = np.load(open("../index/v2/user_queries.npy", "rb"))

kmeans.train(X.astype(np.float32))

print(kmeans.centroids.shape)
# (3578, 512)

#### STEP 2: Extend our queries Graph (Offline)

In [ ]:
import pandas as pd
import Levenshtein as ln
import networkx as nx
from ast import literal_eval

from tqdm import tqdm
import pickle

# let's load v1 data

G = nx.read_gpickle("../index/query_graph.gpickle")
track_leafs = pickle.load(open("../index/track_leafs.pkl", "rb"))
roots_list = [i for i,j in G.nodes(data="starts", default=1) if j==True]

ids_question_map = pickle.load(open("../index/ids_question_map.pkl", "rb"))
question_ids_map = pickle.load(open("../index/question_ids_map.pkl", "rb"))

In [ ]:
import csv
import sys
csv.field_size_limit(sys.maxsize)

old_lines = len(ids_question_map)
lines = len(ids_question_map)
with open("../data/queries.train.tsv", "r") as f:
    tsv_reader = csv.reader(f, delimiter="\t")
    for row in tsv_reader:
        ids_question_map[lines] = row[-1].lower()
        question_ids_map[row[-1].lower()] = lines
        lines += 1
lines - old_lines

In [ ]:

with open("../data/queries.train.tsv", "r") as f:
    tsv_reader = csv.reader(f, delimiter="\t")
    nlines = 0 
    for row in tqdm(tsv_reader, total=808731):
        nlines+=1
        line = row[-1]
        if line!='':
            question = line.strip("\n").lower()
            tokenized = question.split()

            if not tokenized:
                continue
            
            prev_node_entry = tokenized[0]
            for i in range(2, len(tokenized)+1):
              # Create Nodes and add property Starts, if it's the start of
              # query
              if len(prev_node_entry.split()) == 1:
                  if prev_node_entry not in G:
                      G.add_node(prev_node_entry, starts=True)
                  else:
                      G.nodes[prev_node_entry]['starts'] = True
              else:
                  if prev_node_entry not in G:
                      G.add_node(prev_node_entry, starts=False)
                  else:
                      G.nodes[prev_node_entry]['starts'] = False

              if len(" ".join(tokenized[:i]).split()) == 1:
                  if " ".join(tokenized[:i]) not in G:
                      G.add_node(" ".join(tokenized[:i]), starts=True)
                  else:
                      G.nodes[" ".join(tokenized[:i])]['starts'] = True
              else:
                  if " ".join(tokenized[:i]) not in G:
                      G.add_node(" ".join(tokenized[:i]), starts=False)
                  else:
                      G.nodes[" ".join(tokenized[:i])]['starts'] = False

              # pre-computing leaf nodes, so that we don't have to 
              # find descendants during inference
              track_leafs[prev_node_entry] = track_leafs.get(prev_node_entry, []) + question_ids_map[question]
              
              # Add connection between sentence[:i] -> sentence[:i+1]
              G.add_edge(prev_node_entry, " ".join(tokenized[:i]))

              prev_node_entry =  " ".join(tokenized[:i])

          track_leafs[question] = track_leafs.get(question, []) + question_ids_map[question]
              

roots_list = [i for i,j in G.nodes(data="starts", default=1) if j==True]

In [ ]:
nx.write_gpickle(G, "../index/v2/query_graph.gpickle")
pickle.dump(track_leafs, open("../index/v2/track_leafs.pkl", "wb"))
pickle.dump(ids_question_map, open("../index/v2/ids_question_map.pkl", "wb"))
pickle.dump(question_ids_map, open("../index/v2/question_ids_map.pkl", "wb"))


#### STEP 3: Inferenece Pipeline

In [ ]:
G = nx.read_gpickle("../index/v2/query_graph.gpickle")
track_leafs = pickle.load(open("../index/v2/track_leafs.pkl", "rb"))
ids_question_map = pickle.load(open("../index/v2/ids_question_map.pkl", "rb"))
question_ids_map = pickle.load(open("../index/v2/question_ids_map.pkl", "rb"))

roots_list = [i for i,j in G.nodes(data="starts", default=1) if j==True]

import timeit


query = "bbilical defn".lower()

tokenized_query = query.split()
len_tokenized_query = len(tokenized_query)
node_list = []

def recurssive_search(comparison_term, G, idx=1, root=False, node_list=[], forked=False):
    new_term = []
    for node in (roots_list if root else G[" ".join(comparison_term.split()[:-1])]):
        
        # don't auto-correct on last-word
        if ((ln.distance(comparison_term, node) <= 2) if idx!=len_tokenized_query-1 else tokenized_query[-1] in node):
            if idx!=len_tokenized_query-1:
                node_list.extend(recurssive_search(node + " " + tokenized_query[idx+1], G, idx+1, root=False, node_list=node_list, forked=True))
            else:
                if node in G and node in track_leafs:
                    new_term.append(node)

              
    if forked:
        if idx==len_tokenized_query-1:
            return [ids_question_map[child] for term in new_term for child in track_leafs[term]]
            
        return ""
    else:
        if len(tokenized_query)==1:
            if idx==len_tokenized_query-1:
                return [ids_question_map[child] for term in new_term for child in track_leafs[term]]
        return node_list

start_time = timeit.default_timer()
suggested_queries = recurssive_search(tokenized_query[0], G, idx=0, root=True)
# again, aprox. 0.07ms even for root query search like 'what' 
print("Total time taken: ", timeit.default_timer()-start_time)

In [ ]:
# Ranking the suggested_queries

from sklearn.metrics.pairwise import cosine_similarity

# ideally, even this would be pre-computed
test_x = get_embedding(suggested_queries)


# aprox. takes less than 0.01s
query_embed = get_embedding(query)
query_len = len(query)
suggested_queries = np.argsort([c_score*(1/(1+ln.distance(query, suggested_queries[pos][:query_len]))) for pos, c_score in enumerate(np.max(cosine_similarity(test_x, kmeans.centroids + query_embed), axis=-1))])[::-1]
# top-10 relevant suggestions
for tag in suggested_queries[:10]:
    print(suggested_queries[tag])

## Part 3: Evaluation
There are many ways of building a search suggest feature that meets the above requirements. In this part, propose evaluation methods to judge a system's performance. 

How do the two systems developed in Part 1 and Part 2 perform on the proposed evaluation method? 

What are ways we can improve the above prototypes?

### Proposed Solution

Couple of Notes:

*   Our Model hasn't been fully trained on the data. We have used only 10% of the MS-MACRO documents and only 2% of the user queries (for ranking). Therefore, the below evaluation should be considered only as a baseline estimate that we correct once our models can fully utilize the data.
*   Evaluation for this is really tricky. It's very easy to fool ourself that the model is not working well based on the evaluations logic. Since autocompletion requires some prior knowledge, both external knowledge (past user queries) and internal knowledge (hints on the partial user query like entities or relevant Noun-Phrases). 
  * For example execting our model to autocomplete when the partial query is 'what is the best' is not the right expectation. A better query that we should expect our model to autocomplete would be 'what is the best way to file taxes in-`, this partial query can be autocompleted if our approach can pick what region the tenant/user resides-in. A more similar general queries would be like- 'what is the difference between AI and-' 

* Due to time constrain and incomplete trained model, we run the evaluation only for 1000 sample queries.

### Big Picture:

*   We will be using the MS-MARCO queries development set to evaluate our model performance.
*   For each queries, we will break them into contiguous partial query and try to auto-complete the rest. 
*   One thing to note is, we consider to predict and evaluate a partial query only if it contains a Noun Phrase.
*   We will use Exact Match and F1 score to compare the autocompleted query and the original query. 
  * If our query is "how long is a flight from chicago to australia" and our partial query is "how long is a flight from chicago". assume the predicted Autocompletion is "how long is a flight from chicago to US". We compute the two metrics by passing the following information: EM("to US", "to australia") and F1("to US", "to australia")  





#### Step 1: Initialize necessary functions

In [ ]:
# Load files

import pandas as pd
import Levenshtein as ln
import networkx as nx
from ast import literal_eval

from tqdm import tqdm
import pickle
import numpy as np
import spacy
nlp = spacy.load('en_core_web_sm')

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import tensorflow_text

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')

tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])


embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

def get_embedding(text):
    if isinstance(text, str):
        return embed([text]).numpy()
    return embed(text).numpy()

get_embedding("Hello World").shape

In [ ]:
# prediction Code

import Levenshtein as ln


def predict_query(query, G, roots_list, ids_question_map, track_leafs):
    tokenized_query = query.split()
    len_tokenized_query = len(tokenized_query)
    node_list = []

    def recurssive_search(comparison_term, G, idx=1, root=False, node_list=[], forked=False):
        new_term = []
        for node in (roots_list if root else G[" ".join(comparison_term.split()[:-1])]):

            # don't auto-correct on last-word
            if ((ln.distance(comparison_term, node) <= 2) if idx != len_tokenized_query - 1 else tokenized_query[
                                                                                                     -1] in node):
                if idx != len_tokenized_query - 1:
                    node_list.extend(recurssive_search(node + " " + tokenized_query[idx + 1], G, idx + 1, root=False,
                                                       node_list=node_list, forked=True))
                else:
                    if node in G and node in track_leafs:
                        new_term.append(node)

        if forked:
            if idx == len_tokenized_query - 1:
                return [ids_question_map[child] for term in new_term for child in track_leafs[term]]

            return ""
        else:
            if len(tokenized_query) == 1:
                if idx == len_tokenized_query - 1:
                    return [ids_question_map[child] for term in new_term for child in track_leafs[term]]
            return node_list

    res = recurssive_search(tokenized_query[0], G, idx=0, root=True)
    return res

In [ ]:
# these functions are heavily influenced by the HF squad_metrics.py script
def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)


#### Step 2: Evaluate v1, where the train data is only MS-Marco docs

In [ ]:
G = nx.read_gpickle("../index/query_graph.gpickle")
track_leafs = pickle.load(open("../index/track_leafs.pkl", "rb"))
roots_list = [i for i,j in G.nodes(data="starts", default=1) if j==True]

ids_question_map = pickle.load(open("../index/ids_question_map.pkl", "rb"))
question_ids_map = pickle.load(open("../index/question_ids_map.pkl", "rb"))

In [ ]:
import csv

limit = 1000
metrics = {
    "EM": {
        "@3": [],
        "@5": [],
        "@10": []
    },
    "F1": {
        "@3": [],
        "@5": [],
        "@10": []
    }
}
with open("../data/queries.dev.tsv", "r") as f:
    tsv_reader = csv.reader(f, delimiter="\t")
    nlines = 0 
    for row in tqdm(tsv_reader):
        nlines += 1
        line = row[-1].lower()
        if line!='':
            question = line
            tokenized_question = line.split()
            doc = nlp(question)
            np_list = [str(token) for token in doc if token.tag_[:2]=="NN"]
            observed_np = False
            for token_idx in range(1, len(tokenized_question)):
                if observed_np or tokenized_question[token_idx-1] in np_list:
                    observed_np = True
                    suggested_queries = predict_query(" ".join(tokenized_question[:token_idx]), G, roots_list, ids_question_map, track_leafs)[:500]

                    if suggested_queries!=[]:
                        for k in [3, 5, 10]:
                            temp_EM, temp_F1 = [0], [0]
                            for q in suggested_queries[:k]:
                                temp_EM.append(compute_exact_match(q, question))
                                temp_F1.append(compute_f1(q, question))
                            metrics['EM']['@'+str(k)] = metrics['EM']['@'+str(k)] + [max(temp_EM)]
                            metrics['F1']['@'+str(k)] = metrics['F1']['@'+str(k)] + [max(temp_F1)]
        if not limit:
            break
        else:
            limit-=1



In [ ]:
#metrics
print("Average Exact Match Metric: ", "\n", "EM@3: "+str(np.average(metrics['EM']['@3'])), "\n", "EM@5: "+str(np.average(metrics['EM']['@5'])), "\n", "EM@10: "+str(np.average(metrics['EM']['@10'])))
print("--------")
print("Average F1 Metric: ", "\n", "F1@3: "+str(np.average(metrics['F1']['@3'])), "\n", "F1@5: "+str(np.average(metrics['F1']['@5'])), "\n", "F1@10: "+str(np.average(metrics['F1']['@10'])))

## Output

"""
Average Exact Match Metric:  
 EM@3: 0.009933774834437087 
 EM@5: 0.011037527593818985 
 EM@10: 0.012141280353200883
--------
Average F1 Metric:  
 F1@3: 0.4550674975638583 
 F1@5: 0.47319383050217734 
 F1@10: 0.4945249375231127
"""

#### Step 3: Evaluate v2, where the train data is MS-Marco docs + MS-Marco queries.
* Note: We are not ranking the final suggestions yet

In [ ]:
G = nx.read_gpickle("../index/v2/query_graph.gpickle")
track_leafs = pickle.load(open("../index/v2/track_leafs.pkl", "rb"))
ids_question_map = pickle.load(open("../index/v2/ids_question_map.pkl", "rb"))
question_ids_map = pickle.load(open("../index/v2/question_ids_map.pkl", "rb"))

roots_list = [i for i,j in G.nodes(data="starts", default=1) if j==True]

In [ ]:
import csv

#limit = 15140
limit = 1000
metrics = {
    "EM": {
        "@3": [],
        "@5": [],
        "@10": []
    },
    "F1": {
        "@3": [],
        "@5": [],
        "@10": []
    }
}
with open("../data/queries.dev.tsv", "r") as f:
    tsv_reader = csv.reader(f, delimiter="\t")
    nlines = 0 
    for row in tqdm(tsv_reader):
        nlines += 1
        line = row[-1].lower()
        if line!='':
            question = line
            tokenized_question = line.split()
            doc = nlp(question)
            np_list = [str(token) for token in doc if token.tag_[:2]=="NN"]
            observed_np = False
            for token_idx in range(1, len(tokenized_question)):
                if observed_np or tokenized_question[token_idx-1] in np_list:
                    observed_np = True
                    suggested_queries = predict_query(" ".join(tokenized_question[:token_idx]), G, roots_list, ids_question_map, track_leafs)[:500]

                    if suggested_queries!=[]:
                        for k in [3, 5, 10]:
                            temp_EM, temp_F1 = [0], [0]
                            for q in suggested_queries[:k]:
                                temp_EM.append(compute_exact_match(q, question))
                                temp_F1.append(compute_f1(q, question))
                            metrics['EM']['@'+str(k)] = metrics['EM']['@'+str(k)] + [max(temp_EM)]
                            metrics['F1']['@'+str(k)] = metrics['F1']['@'+str(k)] + [max(temp_F1)]
        if not limit:
            break
        else:
            limit-=1

In [ ]:
#metrics
print("Average Exact Match Metric: ", "\n", "EM@3: "+str(np.average(metrics['EM']['@3'])), "\n", "EM@5: "+str(np.average(metrics['EM']['@5'])), "\n", "EM@10: "+str(np.average(metrics['EM']['@10'])))
print("--------")
print("Average F1 Metric: ", "\n", "F1@3: "+str(np.average(metrics['F1']['@3'])), "\n", "F1@5: "+str(np.average(metrics['F1']['@5'])), "\n", "F1@10: "+str(np.average(metrics['F1']['@10'])))

## Output

"""
Average Exact Match Metric:  
 EM@3: 0.009149130832570906 
 EM@5: 0.010064043915827997 
 EM@10: 0.011893870082342177
--------
Average F1 Metric:  
 F1@3: 0.4879684855782274 
 F1@5: 0.5054822473192558 
 F1@10: 0.5287107502350595
"""

#### Step 3: Evaluate v2 + ranking

In [ ]:
import faiss

X = np.load(open("../index/v2/user_queries.npy", "rb"))

D = 512
K = 10
kmeans = faiss.Kmeans(d=D, k=round(16*(X.shape[0]**(1/2))), niter=20, verbose=True, gpu=True)

kmeans.train(X.astype(np.float32))

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

#limit = 15140
limit = 1000
metrics = {
    "EM": {
        "@3": [],
        "@5": [],
        "@10": []
    },
    "F1": {
        "@3": [],
        "@5": [],
        "@10": []
    }
}
with open("../data/queries.dev.tsv", "r") as f:
    tsv_reader = csv.reader(f, delimiter="\t")
    nlines = 0 
    for row in tqdm(tsv_reader):
        nlines += 1
        line = row[-1].lower()
        if line!='':
            question = line
            tokenized_question = line.split()
            if tokenized_question!=[]:
                query_embed = get_embedding(question)
                
                doc = nlp(question)
                np_list = [str(token) for token in doc if token.tag_[:2]=="NN"]
                observed_np = False
                for token_idx in range(1, len(tokenized_question)):
                    if observed_np or tokenized_question[token_idx-1] in np_list:
                        observed_np = True
                        suggested_queries = predict_query(" ".join(tokenized_question[:token_idx]), G, roots_list, ids_question_map, track_leafs)[:500]

                        if suggested_queries!=[]:
                            test_x = get_embedding(suggested_queries)
                            query_len = len(" ".join(tokenized_question[:token_idx]))
                            sorted_res = np.argsort([c_score*(1/(1+ln.distance(" ".join(tokenized_question[:token_idx]), suggested_queries[pos][:query_len]))) for pos, c_score in enumerate(np.max(cosine_similarity(test_x, kmeans.centroids + query_embed), axis=-1))])[::-1]
                            for k in [3, 5, 10]:
                                temp_EM, temp_F1 = [0], [0]
                                filtered_queries = [suggested_queries[tag] for tag in sorted_res[:k]]
                                for q in filtered_queries[:k]:
                                    temp_EM.append(compute_exact_match(q, question))
                                    temp_F1.append(compute_f1(q, question))
                                metrics['EM']['@'+str(k)] = metrics['EM']['@'+str(k)] + [max(temp_EM)]
                                metrics['F1']['@'+str(k)] = metrics['F1']['@'+str(k)] + [max(temp_F1)]
        if not limit:
            break
        else:
            limit-=1

In [ ]:
#metrics
print("Average Exact Match Metric: ", "\n", "EM@3: "+str(np.average(metrics['EM']['@3'])), "\n", "EM@5: "+str(np.average(metrics['EM']['@5'])), "\n", "EM@10: "+str(np.average(metrics['EM']['@10'])))
print("--------")
print("Average F1 Metric: ", "\n", "F1@3: "+str(np.average(metrics['F1']['@3'])), "\n", "F1@5: "+str(np.average(metrics['F1']['@5'])), "\n", "F1@10: "+str(np.average(metrics['F1']['@10'])))

# Output

"""
Average Exact Match Metric:  
 EM@3: 0.013723696248856358 
 EM@5: 0.013723696248856358 
 EM@10: 0.013723696248856358
--------
Average F1 Metric:  
 F1@3: 0.5759839235621964 
 F1@5: 0.58476066024534 
 F1@10: 0.5911137329097126
"""

### Observations


*   We can see that our model has improved over each iteration. Training the model on full dataset would definitely improve it even more and give a good estimate of how well the approach performs.

### Improvements to the protype

* Eventhough I believe our graph could theoretically scale very well. The space complexity is comparitively high. Finding to make use of character-wise or word-wise query graph would be useful. 
* Our current graph traversal doesn't take into consideration, the semantic similarities of tokens. It would be good experiment to think this through and evaluate whether such a improvement is necessary. 
* We have used a lot of extra files to trace some information. This decision was taken due to limitation of in-memory graph. Moving to a distributed graph storage like Dgraph would help us store lot more information in the graph.
* Eventhough we haven't explored Auto-regressive DL models in this prototype, it would definitely be in the next step to improve the approach. We have both user queries and artificially generated queries. Training a Gpt-2 like model to auto-complete query would be a good addition. 

## For Demo, please visit [here](http://cde01575bbdf.ngrok.io/)
